## Topics in this Notebook

[1-7]. Covered earlier 
8. Train Q-learning Agent with Python - Reinforcement Learning Code Project
9. Watch Q-learning Agent Play Game with Python - Reinforcement Learning Code Project
10. Deep Q-Learning - Combining Neural Networks and Reinforcement Learning
11. Replay Memory Explained - Experience for Deep Q-Network Training
12. Training a Deep Q-Network - Reinforcement Learning
13. Training a Deep Q-Network with Fixed Q-targets - Reinforcement Learning

## 7. OpenAI Gym and Python for Q-learning - Reinforcement Learning Code Project

### OpenAI Gym 
Gym has a ton of environments ranging from simple text based games to Atari games like Breakout and Space Invaders. The library is intuitive to use and simple to install. Just run pip install gym, and you’re good to go! The link to Gym’s installation instructions, requirements, and documentation is included in the description. Go ahead and get that installed now because we'll need it in just a moment.

We’ll be making use of Gym to provide us with an environment for a simple game called Frozen Lake. We’ll then train an agent to play the game using Q-learning, and we’ll get a playback of how the agent does after being trained. 

### Frozen Lake
 I’ve grabbed the description of the game directly from Gym’s website. Let’s read through it together.

Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:

            SFFF
            FHFH
            FFFH
            HFFG

This grid is our environment where **S** is the agent’s starting point, and it’s safe. **F** represents the frozen surface and is also safe. **H** represents a hole, and if our agent steps in a hole in the middle of a frozen lake, well, that’s not good. Finally, **G** represents the goal, which is the space on the grid where the prized frisbee is located.

The agent can navigate left, right, up, and down, and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of one if it reaches the goal, and zero otherwise.

| State | Description | Reward |  
| --- | --- | --- |  
| S | Agent’s starting point - safe | 0 |  
| F | Frozen surface - safe | 0 |  
| H | Hole - game over | 0 |  
| G | Goal - game over | 1 |  

Alright, so you got it? Our agent has to navigate the grid by staying on the frozen surface without falling into any holes until it reaches the frisbee. If it reaches the frisbee, it wins with a reward of plus one. If it falls in a hole, it loses and receives no points for the entire episode.

#### Setting up Frozen Lake in code
**Libraries**

First we’re importing all the libraries we’ll be using. Not many, really... *Numpy*, *gym*, *random*, *time*, and *clear_output* from *Ipython’s display*.


**Creating the environment**

Next, to create our environment, we just call gym.make() and pass a string of the name of the environment we want to set up. We'll be using the environment FrozenLake-v0. All the environments with their corresponding names you can use here are available on Gym’s website.

With this *env* object, we’re able to query for information about the environment, sample states and actions, retrieve rewards, and have our agent navigate the frozen lake. That’s all made available to us conveniently with Gym.

**Creating the Q-table**

We’re now going to construct our Q-table, and initialize all the Q-values to zero for each state-action pair.

Remember, the number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space. We can get this information using using *env.observation_space.n* and *env.action_space.n*, as shown below. We can then use this information to build the Q-table and fill it with zeros.


If you're foggy about Q-tables at all, be sure to check out the earlier post where we covered all the details you need for Q-tables.

Alright, here’s our Q-table!

**Initializing Q-learning parameters**

Now, we’re going to create and initialize all the parameters needed to implement the Q-learning algorithm.

Let's step through each of these.

First, with *num_episodes*, we define the total number of episodes we want the agent to play during training. Then, with *max_steps_per_episode*, we define a maximum number of steps that our agent is allowed to take within a single episode. So, if by the one-hundredth step, the agent hasn’t reached the frisbee or fallen through a hole, then the episode will terminate with the agent receiving zero points.

Next, we set our learning_rate, which was mathematically shown using the symbol $α$ in the previous post. Then, we also set our discount_rate, as well, which was represented with the symbol $γ$ previously.

Now, the last four parameters are all for related to the exploration-exploitation trade-off we talked about last time in regards to the epsilon-greedy policy. We’re initializing our exploration_rate to 1 and setting the *max_exploration_rate* to 1 and a *min_exploration_rate* to 0.01. The max and min are just bounds to how large or small our exploration rate can be. Remember, the exploration rate was represented with the symbol $ϵ$ when we discussed it previously.

Lastly, we set the *exploration_decay_rate* to $0.01$ to determine the rate at which the *exploration_rate* will decay.

Now, all of these parameters can change. These are the parameters you’ll want to play with and tune yourself to see how they influence and change the performance of the algorithm. 

### Code Summarization

In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

env = gym.make("FrozenLake-v0")

action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

print(q_table)

num_episodes = 10000
max_steps_per_episode = 100


learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### 8. Coding the Q-learning algorithm training loop
First, we create this list to hold all of the rewards we’ll get from each episode. This will be so we can see how our game score changes over time. We’ll discuss this more in a bit.

In the following block of code, we’ll implement the entire Q-learning algorithm we discussed in detail in a couple posts back. When this code is executed, this is exactly where the training will take place. This first for-loop contains everything that happens within a single episode. This second nested loop contains everything that happens for a single time-step. 

 **For each episode**
Let's get inside of our first loop. For each episode, we’re going to first reset the state of the environment back to the starting state. 

The *done* variable just keeps track of whether or not our episode is finished, so we initialize it to *False* when we first start the episode, and we'll see later where it will get updated to notify us when the episode is over.

Then, we need to keep track of the rewards within the current episode as well, so we set *rewards_current_episode* to *0* since we start out with no rewards at the beginning of each episode.

**For each time-step**

Now we're entering into the nested loop, which runs for each time-step within an episode. The remaining steps, until we say otherwise, will occur for each time-step.

**Exploration vs. exploitation**

For each time-step within an episode, we set our *exploration_rate_threshold* to a random number between *0* and *1*. This will be used to determine whether our agent will explore or exploit the environment in this time-step, and we discussed the detail of this exploration-exploitation trade-off in a previous post of this series.

If the threshold is greater than the *exploration_rate*, which remember, is initially set to *1*, then our agent will exploit the environment and choose the action that has the highest Q-value in the Q-table for the current state. If, on the other hand, the threshold is less than or equal to the *exploration_rate*, then the agent will explore the environment, and sample an action randomly.

**Taking action**

After our action is chosen, we then take that action by calling step() on our env object and passing our action to it. The function step() returns a tuple containing the new state, the reward for the action we took, whether or not the action ended our episode, and diagnostic information regarding our environment, which may be helpful for us if we end up needing to do any debugging.

**Update the Q-value**

After we observe the reward we obtained from taking the action from the previous state, we can then update the Q-value for that state-action pair in the Q-table. This is done using the formula we introduced in an earlier post, and remember, there we walked through a concrete example showing how to implement the Q-table update.

Here is the forumla:
$$qnew(s,a)=(1−α) q(s,a)old value+α learned value(Rt+1+γmaxa′q(s′,a′))

And here is the same formula in code: 

So, remember, the new Q-value for this state-action pair is a weighted sum of our old value and the “learned value.” So we have our new Q-value equal to the old Q-value times one minus the learning rate plus the learning rate times the “learned value,” which is the reward we just received from our last action plus the discounted estimate of the optimal future Q-value for the next state action pair.

**Transition to the next state**

Next, we set our current state to the new_state that was returned to us once we took our last action, and we then update the rewards from our current episode by adding the reward we received for our previous action.


We then check to see if our last action ended the episode for us, meaning, did our agent step in a hole or reach the goal? If the action did end the episode, then we jump out of this loop and move on to the next episode. Otherwise, we transition to the next time step.  
 
**Exploration rate decay**

Once an episode is finished, we need to update our exploration_rate using exponential decay, which just means that the exploration rate decreases or decays at a rate proportional to its current value. We can decay the exploration_rate using the formula above, which makes use of all the exploration rate parameters that we defined last time. 

 We then just append the rewards from the current episode to the list of rewards from all episodes, and that’s it! We’re good to move on to the next episode.
 
**After all episodes complete**

After all episodes are finished, we now just calculate the average reward per thousand episodes from our list that contains the rewards for all episodes so that we can print it out and see how the rewards changed over time. 

From this printout, we can see our average reward per thousand episodes did indeed progress over time. When the algorithm first started training, the first thousand episodes only averaged a reward of 0.16, but by the time it got to its last thousand episodes, the reward drastically improved to 0.7.
Interpreting the training results

Let’s take a second to understand how we can interpret these results. Our agent played 10,000 episodes. At each time step within an episode, the agent received a reward of 1 if it reached the frisbee, otherwise, it received a reward of 0. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

So, that means for each episode, the total reward received by the agent for the entire episode is either 1 or 0. So, for the first thousand episodes, we can interpret this score as meaning that 16%
of the time, the agent received a reward of 1 and won the episode. And by the last thousand episodes from a total of 10,000, the agent was winning $70%$ of the time.

By analyzing the grid of the game, we can see it’s a lot more likely that the agent would fall in a hole or perhaps reach the max time steps than it is to reach the frisbee, so reaching the frisbee 70%
of the time isn’t too shabby, especially since the agent had no explicit instructions to reach the frisbee. It learned that this is the correct thing to do. 

        SFFF
        FHFH
        FFFH
        HFFG

Lastly, we print out our updated Q-table to see how that has transitioned from its initial state of all zeros. 

### Code Summarization

In [2]:
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode): 

        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
        
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Set new state
        state = new_state
        
        # Add new reward
        rewards_current_episode += reward 
        
        if done == True: 
            break
        
        
    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)
    
    

In [3]:
# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)

********Average reward per thousand episodes********

1000 :  0.04000000000000003
2000 :  0.20600000000000016
3000 :  0.4250000000000003
4000 :  0.5350000000000004
5000 :  0.6230000000000004
6000 :  0.6820000000000005
7000 :  0.6530000000000005
8000 :  0.6760000000000005
9000 :  0.6750000000000005
10000 :  0.7080000000000005


********Q-table********

[[0.55205409 0.50448733 0.50494711 0.50274407]
 [0.30432154 0.3711061  0.21238507 0.50276836]
 [0.39792925 0.44036372 0.38494437 0.47331413]
 [0.21281496 0.29848727 0.14125936 0.45602412]
 [0.57017843 0.36751775 0.3576583  0.38019348]
 [0.         0.         0.         0.        ]
 [0.34698309 0.1052313  0.17366741 0.15303588]
 [0.         0.         0.         0.        ]
 [0.36275685 0.3648826  0.28670045 0.60254228]
 [0.39313746 0.65694069 0.41143031 0.34838357]
 [0.63603086 0.38406221 0.2770558  0.27242499]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.49801306 0.49629306 0.71002056 

## 9. Watch Q-learning Agent Play Game with Python - Reinforcement Learning Code Project

### The code to watch the agent play the game

This block of code is going to allow us to watch our trained agent play Frozen Lake using the knowledge it’s gained from the training we completed. 

We're going to watch the agent play three episodes. Let's look at the start of the outter loop first.

#### For each episode 

For each of the three episodes, we first reset the state of our environment, and set done to False. This variable is used for the same purpose as we saw in our training loop last time. It just keeps track whether or not our last action ended the episode.

We then just print to the console what episode we’re starting, and we sleep for one second so that we have time to actually read that printout before it disappears from the screen.

Now, we'll move on to the inner loop.

#### For each time-step 

For each time-step within the episode, we’re calling the `iPython display` function `clear_output()`, which clears the output from the current cell in the Jupyter notebook. With `wait=True`, it waits to clear the output until there is another printout to overwrite it. This is all just done so that the notebook and the screen display remain smooth as we watch our agent play.

We then call `render()` on our `env` object, which will render the current state of the environment to the display so that we can actually visually see the game grid and where our agent is on the grid. We then `sleep` again for `300` milliseconds to give us time to actually see the current state of the environment on screen before moving on to the next time step. Don't worry, this will all come together once we view the final product. 

We then set our `action` to be the action that has the highest Q-value from our Q-table for our current `state`, and then we take that action with `env.step()`, just like we saw during training. This will update our `new_state`, the `reward` for our `action`, and whether or not the action completed the episode. 

If our `action` did end the episode, then we `render` the environment to see where the agent ended up from our last time-step. If the `reward` for that `action` was a `1`, then we know that the episode ended because the agent reached the frisbee and won the game. So we print that info to the console. If the reward wasn’t a `1`, then we know it was alternatively a `0` and that the agent fell through a hole.

After seeing how the episode ended, we then start a new episode.

Now, if the last action didn’t complete the episode, then we skip over the conditional, transition to the new state, and move on to the next time step. 

After all three episodes are done, we then close the environment, and that’s it! 

### Code Summarization

In [4]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):       
        
        # Show current state of environment on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        # Choose action with highest Q-value for current state  
        action = np.argmax(q_table[state,:])
        
        # Take new action
        new_state, reward, done, info = env.step(action)
             
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                # Agent stepped in a hole and lost episode
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
            
        # Set new state
        state = new_state
        
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****


### Watching the agent play

Alright, now in the video, we run this code and watch the agent play! Here's what we expect. We’ll have our grid printed to the screen, the agent will start in the starting state in the top left corner of the grid, and we’ll be able to see the actions chosen by the agent displayed above the grid at each time step. We’ll also see the agent move around the grid, as indicated with a red marker.

Remember when we introduced Frozen Lake, part of the description noted that the agent won’t always take the action that it chooses to take because, since the ice is slippery, even if we choose to go right, for example, we may slip and go up instead. So keep this in mind as you watch the agent play because you may see the chosen action show as right but then see the agent take a step up, for example. The slippery ice is the reason for this.

Now check out the agent play in the video!


## Putting it all together

In [ ]:
%%writefile OpenGymFrozenLake_QLearning.py

## Playing FrozenLake using Q-Learning 

## Loading Libraries
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

env = gym.make("FrozenLake-v0")

action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

print(q_table)

num_episodes = 10000
max_steps_per_episode = 100


learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001


rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode): 

        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
        
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Set new state
        state = new_state
        
        # Add new reward
        rewards_current_episode += reward 
        
        if done == True: 
            break
        
    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)


    # Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)


# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table
for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):       
        
        # Show current state of environment on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        # Choose action with highest Q-value for current state  
        action = np.argmax(q_table[state,:])
        
        # Take new action
        new_state, reward, done, info = env.step(action)
             
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                # Agent stepped in a hole and lost episode
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
            
        # Set new state
        state = new_state
        
env.close()
